In [38]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy

from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset, Subset

from utils import metrics


In [39]:
def predict_all_visits_bce_loss(y_pred, y_true, x_lab_length):
    batch_size = len(y_true)
    loss = torch.nn.BCELoss()
    indices = torch.arange(batch_size, dtype=torch.int64)
    losses = 0
    for i in indices:
        visits_indices = torch.arange(x_lab_length[i], dtype=torch.int64)
        patient_losses = 0
        for v in visits_indices:
            patient_losses += loss(y_pred[i][v], y_true[i][v])
        losses += patient_losses/x_lab_length[i]
    return losses/batch_size

In [40]:
class Dataset(data.Dataset):
    def __init__(self, x, y, x_lab_length):
        self.x= x
        self.y = y
        self.x_lab_length = x_lab_length
        
    def __getitem__(self, index): # 返回的是tensor
        return self.x[index], self.y[index], self.x_lab_length[index]

    def __len__(self):
        return len(self.y)

In [41]:
data_path = '../dataset/tongji/processed_data/'
file_name = './ckpt/gru.pth'

batch_size = 64
num_epochs = 50
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cpu')

data_path = "../dataset/tongji/processed_data/"
x = pickle.load(open(data_path+"x.pkl", "rb"))
y = pickle.load(open(data_path+"y.pkl", "rb"))
x_lab_length = pickle.load(open(data_path+"visits_length.pkl", "rb"))
train_dataset = Dataset(x, y, x_lab_length)


In [42]:
class GRU(nn.Module):
    def __init__(self, input_demo_dim, input_lab_dim, max_visits, hidden_dim, output_dim, act_layer=nn.GELU, drop=0.):
        super(GRU, self).__init__()

        # hyperparameters
        self.input_demo_dim = input_demo_dim
        self.input_lab_dim = input_lab_dim
        self.max_visits = max_visits
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.proj = nn.Linear(input_demo_dim+input_lab_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(max_visits)
        self.gru = nn.GRU(input_size = hidden_dim, hidden_size = hidden_dim, num_layers = 1, batch_first = True)
        
        self.act = act_layer()
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.drop = nn.Dropout(drop)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.proj(x)
        # x = self.act(x)
        # x = self.bn(x)
        x, _ = self.gru(x) # x: (batch_size,L,hidden_dim) h_n: (1, batch_size, hidden_dim)
        x = self.drop(x)
        x = self.fc(x)
        x = self.drop(x)
        x = self.sigmoid(x)
        return x

In [43]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) # numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) # gpu
torch.backends.cudnn.deterministic=True # cudnn
np.set_printoptions(threshold=np.inf, precision=2, suppress=True)


def train_epoch(model, device, dataloader, loss_fn, optimizer):
    train_loss = []
    model.train()
    for step, data in enumerate(dataloader):   
        batch_x, batch_y, batch_x_lab_length = data
        batch_x, batch_y, batch_x_lab_length = batch_x.float(), batch_y.float(), batch_x_lab_length.float()
        batch_y = batch_y[:,:,0] # 0: outcome, 1: los
        batch_y = batch_y.unsqueeze(-1)
        optimizer.zero_grad()
        output = model(batch_x)
        loss = loss_fn(output, batch_y, batch_x_lab_length)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    return np.array(train_loss).mean()

def val_epoch(model, device, dataloader, loss_fn):
    """
    val / test
    """
    val_loss = []
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):   
            batch_x, batch_y, batch_x_lab_length = data
            batch_x, batch_y, batch_x_lab_length = batch_x.float(), batch_y.float(), batch_x_lab_length.float()
            batch_y = batch_y[:,:,0] # 0: outcome, 1: los
            batch_y = batch_y.unsqueeze(-1)
            output = model(batch_x)
            loss = loss_fn(output, batch_y, batch_x_lab_length)
            val_loss.append(loss.item())
            for i in range(len(batch_y)):
                y_pred.extend(output[i][:batch_x_lab_length[i].long()].tolist())
                y_true.extend(batch_y[i][:batch_x_lab_length[i].long()].tolist())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    evaluation_scores = metrics.print_metrics_binary(y_true, y_pred)
    return np.array(val_loss).mean(), evaluation_scores


In [44]:
model = GRU(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = predict_all_visits_bce_loss
train_loader = DataLoader(train_dataset, batch_size=128)
train_loss = train_epoch(model, device, train_loader, criterion, optimizer)
val_loss = val_epoch(model, device, train_loader, criterion)

confusion matrix:
[[1004   46]
 [ 332  322]]
accuracy = 0.7781690359115601
precision class 0 = 0.7514970302581787
precision class 1 = 0.875
recall class 0 = 0.9561904668807983
recall class 1 = 0.49235475063323975
AUC of ROC = 0.8350924712392603
AUC of PRC = 0.7870951405549289
min(+P, Se) = 0.6931297709923664
f1_score = 0.6301369949598974


In [45]:
all_history={}
test_performance={'test_loss': [],'test_accuracy':[], 'test_auroc':[], 'test_auprc':[]}
dataset = train_dataset
num_folds = 10
kfold_test = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_SEED)
skf = kfold_test.split(np.arange(len(dataset)), dataset.y[:, 0, 0])
for fold_test, (train_and_val_idx, test_idx) in enumerate(skf):
    print('====== Test Fold {} ======'.format(fold_test + 1))
    sss = StratifiedShuffleSplit(
        n_splits=1, test_size=1 / (num_folds - 1), random_state=RANDOM_SEED
    )

    test_sampler = SubsetRandomSampler(test_idx)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
    sub_dataset = Dataset(dataset.x[train_and_val_idx], dataset.y[train_and_val_idx], dataset.x_lab_length[train_and_val_idx])
    all_history['test_fold_{}'.format(fold_test+1)] = {}

    train_idx, val_idx = next(
        sss.split(np.arange(len(train_and_val_idx)), sub_dataset.y[:, 0, 0])
    )

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
    model = GRU(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = predict_all_visits_bce_loss
    history = {'train_loss': [], 'val_loss': [],'val_accuracy':[], 'val_auroc':[], 'val_auprc':[]}
    best_val_performance = 0.0
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, device, train_loader, criterion, optimizer)
        val_loss, val_evaluation_scores=val_epoch(model, device, val_loader, criterion)
        # save performance history on validation set
        print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Val Loss:{:.3f}".format(epoch + 1, num_epochs, train_loss, val_loss))
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_evaluation_scores['acc'])
        history['val_auroc'].append(val_evaluation_scores['auroc'])
        history['val_auprc'].append(val_evaluation_scores['auprc'])
        all_history['test_fold_{}'.format(fold_test+1)] = history

        # if auroc is better, than set the best auroc, save the model, and test it on the test set
        if val_evaluation_scores['auroc'] > best_val_performance:
            best_val_performance = val_evaluation_scores['auroc']
            torch.save(model.state_dict(), './checkpoints/gru_best_auroc.pth')
            
    print(
        f"Best performance on val set {fold_test+1}: \
        AUROC = {best_val_performance}"
    )
    model = GRU(input_lab_dim=25, input_demo_dim=2, max_visits=13, hidden_dim=32, output_dim=1, act_layer=nn.GELU, drop=0.).to(device)
    model.load_state_dict(torch.load('./checkpoints/gru_best_auroc.pth'))
    test_loss, test_evaluation_scores = val_epoch(model, device, test_loader, criterion)
    test_performance['test_loss'].append(test_loss)
    test_performance['test_accuracy'].append(test_evaluation_scores['acc'])
    test_performance['test_auroc'].append(test_evaluation_scores['auroc'])
    test_performance['test_auprc'].append(test_evaluation_scores['auprc'])
    print(f"Performance on test set {fold_test+1}: Accuracy = {test_evaluation_scores['acc']}, AUROC = {test_evaluation_scores['auroc']}, AUPRC = {test_evaluation_scores['auprc']}")
        
    


====== Test Fold 1 ======
confusion matrix:
[[102   5]
 [ 29  30]]
accuracy = 0.7951807379722595
precision class 0 = 0.7786259651184082
precision class 1 = 0.8571428656578064
recall class 0 = 0.9532710313796997
recall class 1 = 0.508474588394165
AUC of ROC = 0.8824647552669095
AUC of PRC = 0.8549289341398226
min(+P, Se) = 0.7796610169491526
f1_score = 0.6382978563936883
Epoch:1/50 AVG Training Loss:0.686 AVG Val Loss:0.653
confusion matrix:
[[107   0]
 [ 21  38]]
accuracy = 0.8734939694404602
precision class 0 = 0.8359375
precision class 1 = 1.0
recall class 0 = 1.0
recall class 1 = 0.6440678238868713
AUC of ROC = 0.9600823697132901
AUC of PRC = 0.951056853035557
min(+P, Se) = 0.864406779661017
f1_score = 0.7835052032275093
Epoch:2/50 AVG Training Loss:0.630 AVG Val Loss:0.597
confusion matrix:
[[107   0]
 [ 18  41]]
accuracy = 0.891566276550293
precision class 0 = 0.8560000061988831
precision class 1 = 1.0
recall class 0 = 1.0
recall class 1 = 0.694915235042572
AUC of ROC = 0.96816093

/home/zhuyh/projects/covid-emr/notebooks/utils/metrics.py:22: RuntimeWarning: invalid value encountered in float_scalars
  prec0 = cf[0][0] / (cf[0][0] + cf[1][0])


confusion matrix:
[[84 26]
 [ 5 56]]
accuracy = 0.8187134265899658
precision class 0 = 0.9438202381134033
precision class 1 = 0.6829268336296082
recall class 0 = 0.7636363506317139
recall class 1 = 0.9180327653884888
AUC of ROC = 0.94575260804769
AUC of PRC = 0.9233012766035514
min(+P, Se) = 0.7868852459016393
f1_score = 0.7832168074212427
Epoch:3/50 AVG Training Loss:0.634 AVG Val Loss:0.614
confusion matrix:
[[99 11]
 [11 50]]
accuracy = 0.871345043182373
precision class 0 = 0.8999999761581421
precision class 1 = 0.8196721076965332
recall class 0 = 0.8999999761581421
recall class 1 = 0.8196721076965332
AUC of ROC = 0.9543964232488823
AUC of PRC = 0.9348388417810125
min(+P, Se) = 0.819672131147541
f1_score = 0.8196721076965332
Epoch:4/50 AVG Training Loss:0.582 AVG Val Loss:0.560
confusion matrix:
[[104   6]
 [ 13  48]]
accuracy = 0.8888888955116272
precision class 0 = 0.8888888955116272
precision class 1 = 0.8888888955116272
recall class 0 = 0.9454545378684998
recall class 1 = 0.7868

In [47]:
# Calculate average performance on 10-fold test set
print(test_performance)
test_accuracy_list = np.array(test_performance['test_accuracy'])
test_auroc_list = np.array(test_performance['test_auroc'])
test_auprc_list = np.array(test_performance['test_auprc'])
print('accuracy: mean={:.3f}, std={:.3f}'.format(test_accuracy_list.mean(), test_accuracy_list.std()))
print('auroc: mean={:.3f}, std={:.3f}'.format(test_auroc_list.mean(), test_auroc_list.std()))
print('auprc: mean={:.3f}, std={:.3f}'.format(test_auprc_list.mean(), test_auprc_list.std()))

"""
accuracy: mean=0.935, std=0.075
auroc: mean=0.978, std=0.047
auprc: mean=0.965, std=0.078
"""

{'test_loss': [0.13519078493118286, 0.10609977692365646, 0.09619823843240738, 0.2789987027645111, 0.12341393530368805, 0.6481584906578064, 0.11375057697296143, 0.059119813144207, 0.05398879945278168, 0.12142005562782288], 'test_accuracy': [0.9479769, 0.96666664, 0.98285717, 0.9205298, 0.9322034, 0.72, 0.96732026, 0.98351645, 0.99418604, 0.93373495], 'test_auroc': [0.9883885584820165, 0.9955191396748175, 0.9991079393398751, 0.9772727272727273, 0.9944193462662769, 0.8370937416062314, 0.9982443820224719, 0.9998666133119914, 0.9998592936541438, 0.9914823914823914], 'test_auprc': [0.9846828218253756, 0.9932396254660512, 0.9982630405534096, 0.9688678511899442, 0.9916400808836439, 0.7323535854185843, 0.9976396398867099, 0.9997500157470396, 0.9997914603774714, 0.9848244704137135]}
accuracy: mean=0.935, std=0.075
auroc: mean=0.978, std=0.047
auprc: mean=0.965, std=0.078


'\naccuracy: mean=0.956, std=0.025\nauroc: mean=0.986, std=0.015\nauprc: mean=0.984, std=0.018\n'